Prior to loading the dataset into R, it was pre-processed in Excel. The raw elemental intensities were each divided by the Rh column, which scaled each element, and a new column for each element was generated. Then, all amounts less than or equal to 0 were replaced with 0.0001, to avoid any issues with NaNs in the dataset.

# Load libraries

In [1]:
# try installing vctrs fresh, since it is giving me errors
#install.packages("vctrs", dependencies = TRUE, repos = 'http://cran.us.r-project.org')
library(vctrs)

# Dicer library is used to run ensemble clustering, which generates interim labels for unknown samples
#install.packages("diceR", dependencies = TRUE, repos = 'http://cran.us.r-project.org')
library(diceR)

# try installing Biobase, since I got error messages
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install("Biobase")
library(Biobase)

# randomForest is the classification system which identifies unknown samples
#install.packages("randomForest", dependencies = TRUE, repos = 'http://cran.us.r-project.org')
library(randomForest)

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To 

# Load train dataset

In [2]:
train <- read.csv("AllSamples.csv", header = TRUE)
head(train)

,X,is_known,Vis,Ag,Al,As,Au,Ca,Cu,Fe,⋯,Si.Rh,Sn.Rh,Sr.Rh,Th.Rh,Ti.Rh,U.Rh,V.Rh,Y.Rh,Zn.Rh,Zr.Rh
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,71.11.1-89.01,Guess,Gravel Cherts,1368,1,0.0001,54,1515,331,5335,⋯,0.06234530,0.030244431,0.13691213,2.328280e-02,0.015702351,0.027769183,0.005491955,0.003635520,0.090423886,7.740000e-09
2,71.11.1-89.02,Guess,Gravel Cherts,1269,15,55.0000,25,682,300,2670,⋯,0.06373444,0.019170124,0.18995851,1.294606e-02,0.013609959,0.016431535,0.006887967,0.000083000,0.071867220,8.300000e-09
3,71.11.1-89.03,Guess,Gravel Cherts,1165,32,4.0000,46,607,523,6119,⋯,0.05545573,0.010944136,0.09882391,7.350539e-03,0.018703038,0.008330611,0.005308723,0.004573669,0.039692911,2.531852e-03
4,71.11.1-89.04,Guess,Gravel Cherts,1299,40,42.0000,45,338,410,5307,⋯,0.06011742,0.010667328,0.04589432,1.174233e-02,0.021582734,0.024559663,0.001901927,0.011080791,0.032746217,4.796163e-03
5,71.11.1-89.05,Guess,Gravel Cherts,1724,35,0.0001,51,201,301,1745,⋯,0.07111721,0.002502422,0.08411366,6.780756e-03,0.010251857,0.010494026,0.005247013,0.012431385,0.009283177,8.070000e-09
6,71.11.1-89.06,Guess,Gravel Cherts,1246,139,159.0000,18,188,292,4858,⋯,0.07108405,0.009646556,0.09575393,7.910000e-09,0.008381434,0.006167471,0.001897683,0.000079100,0.014627975,5.851190e-03


In [3]:
# restrict dataset to only important columns: sample id, is_known, and the elemental intensities
train_df = train[,c(1,2,3,32:58)]
# rename X to sample_id
names(train_df)[1] <- "sample_id"
# preview dataset
head(train_df)

,sample_id,is_known,Vis,Ag.Rh,Al.Rh,As.Rh,Au.Rh,Ca.Rh,Cu.Rh,Fe.Rh,⋯,Si.Rh,Sn.Rh,Sr.Rh,Th.Rh,Ti.Rh,U.Rh,V.Rh,Y.Rh,Zn.Rh,Zr.Rh
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,71.11.1-89.01,Guess,Gravel Cherts,0.10581683,0.000077400,7.740000e-09,0.004176980,0.11718750,0.02560334,0.4126702,⋯,0.06234530,0.030244431,0.13691213,2.328280e-02,0.015702351,0.027769183,0.005491955,0.003635520,0.090423886,7.740000e-09
2,71.11.1-89.02,Guess,Gravel Cherts,0.10531120,0.001244813,4.564315e-03,0.002074689,0.05659751,0.02489627,0.2215768,⋯,0.06373444,0.019170124,0.18995851,1.294606e-02,0.013609959,0.016431535,0.006887967,0.000083000,0.071867220,8.300000e-09
3,71.11.1-89.03,Guess,Gravel Cherts,0.09514864,0.002613525,3.266910e-04,0.003756942,0.04957530,0.04271480,0.4997550,⋯,0.05545573,0.010944136,0.09882391,7.350539e-03,0.018703038,0.008330611,0.005308723,0.004573669,0.039692911,2.531852e-03
4,71.11.1-89.04,Guess,Gravel Cherts,0.10741751,0.003307699,3.473084e-03,0.003721161,0.02795005,0.03390391,0.4388489,⋯,0.06011742,0.010667328,0.04589432,1.174233e-02,0.021582734,0.024559663,0.001901927,0.011080791,0.032746217,4.796163e-03
5,71.11.1-89.05,Guess,Gravel Cherts,0.13916694,0.002825315,8.070000e-09,0.004116887,0.01622538,0.02429771,0.1408621,⋯,0.07111721,0.002502422,0.08411366,6.780756e-03,0.010251857,0.010494026,0.005247013,0.012431385,0.009283177,8.070000e-09
6,71.11.1-89.06,Guess,Gravel Cherts,0.09852139,0.010990749,1.257215e-02,0.001423262,0.01486518,0.02308848,0.3841227,⋯,0.07108405,0.009646556,0.09575393,7.910000e-09,0.008381434,0.006167471,0.001897683,0.000079100,0.014627975,5.851190e-03


# Run ensemble clustering on train dataset

For this portion we used diceR to run a consensus cluster with the following settings:
- nk = 2 - 15 clusters, to be sure I captured all possible groupings
- reps = 1 because I found that multiple attempts to cluster the same way didn't change the result significantly
- We also broke up the number of clusters to alleviate processing time, so a max of 4 algorithms are run in each command.

In [4]:
cluster1 <- consensus_cluster(train_df[,4:30], nk=2:15, p.item=1, reps=1, 
                             algorithms=c("nmf", "hc", "diana", "km"), scale = FALSE)

New names:
* Ag.Rh -> Ag.Rh...1
* Al.Rh -> Al.Rh...2
* As.Rh -> As.Rh...3
* Au.Rh -> Au.Rh...4
* Ca.Rh -> Ca.Rh...5
* ...


Clustering Algorithm 1 of 5: NMF_Brunet (k = 3) [--------------]   3% eta:  4m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 4) [>-------------]   4% eta: 10m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 5) [>-------------]   6% eta: 12m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 6) [>-------------]   7% eta: 11m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 7) [>-------------]   9% eta: 13m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 8) [>-------------]  10% eta: 13m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 9) [=>------------]  11% eta: 14m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 10) [=>-----------]  13% eta: 14m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 11) [=>-----------]  14% eta: 14m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 12) [=>-----------]  16% eta: 15m

Clustering Algorithm 1 of 5: NMF_Brunet (k = 13) [=>-----------]  17% eta: 1

In [5]:
cluster2 <- consensus_cluster(train_df[,4:30], nk=2:15, p.item=1, reps=1, 
                             algorithms=c("pam", "ap", "sc", "gmm"), scale = FALSE)


Clustering Algorithm 1 of 4: PAM_Euclidean (k = 4) [>----------]   5% eta:  6s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 5) [>----------]   7% eta:  9s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 6) [>----------]   9% eta: 10s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 7) [>----------]  11% eta: 10s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 8) [>----------]  12% eta: 12s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 9) [=>---------]  14% eta: 14s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 10) [=>--------]  16% eta: 16s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 11) [=>--------]  18% eta: 18s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 12) [=>--------]  20% eta: 20s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 13) [=>--------]  21% eta: 24s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 14) [=>--------]  23% eta: 27s

Clustering Algorithm 1 of 4: PAM_Euclidean (k = 15) [=>--------]  25% eta: 29s

Clustering Algorithm 2 of 4: AP (k = 2)

In [6]:
cluster3 <- consensus_cluster(train_df[,4:30], nk=2:15, p.item=1, reps=1, 
                             algorithms=c("block", "som", "cmeans"), scale = FALSE)


Clustering Algorithm 1 of 3: BLOCK (k = 3) [>------------------]   5% eta:  4s

Clustering Algorithm 1 of 3: BLOCK (k = 4) [>------------------]   7% eta:  7s

Clustering Algorithm 1 of 3: BLOCK (k = 5) [=>-----------------]  10% eta:  8s

Clustering Algorithm 1 of 3: BLOCK (k = 6) [=>-----------------]  12% eta: 10s

Clustering Algorithm 1 of 3: BLOCK (k = 7) [==>----------------]  14% eta: 16s

Clustering Algorithm 1 of 3: BLOCK (k = 8) [==>----------------]  17% eta: 21s

Clustering Algorithm 1 of 3: BLOCK (k = 9) [===>---------------]  19% eta: 31s

Clustering Algorithm 1 of 3: BLOCK (k = 10) [===>--------------]  21% eta: 41s

Clustering Algorithm 1 of 3: BLOCK (k = 11) [===>--------------]  24% eta:  1m

Clustering Algorithm 1 of 3: BLOCK (k = 12) [====>-------------]  26% eta:  1m

Clustering Algorithm 1 of 3: BLOCK (k = 13) [====>-------------]  29% eta:  1m

Clustering Algorithm 1 of 3: BLOCK (k = 14) [=====>------------]  31% eta:  1m

Clustering Algorithm 1 of 3: BLOCK (k =

# Choose a clustering method

In this section we examine each of the results of the clustering algorithms and choose that which best groups our 'known' groups together and separates them out. Before this can be done, however, we need to do some work turning the results of the clustering step into readable data frames.

In [7]:
# save the results of the algorithm as individual dataframes
brunet <- cluster1[,,"NMF_Brunet",1:14]
lee <- cluster1[,,"NMF_Lee",1:14]
hc <- cluster1[,,"HC_Euclidean",1:14]
diana <- cluster1[,,"DIANA_Euclidean",1:14]
km <- cluster1[,,"KM",1:14]
pam <- cluster2[,,"PAM_Euclidean",1:14]
ap <- cluster2[,,"AP",1:14]
sc <- cluster2[,,"SC",1:14]
gmm <- cluster2[,,"GMM",1:14]
block <- cluster3[,,"BLOCK",1:14]
som <- cluster3[,,"SOM",1:14]
cmeans <- cluster3[,,"CMEANS",1:14]

In [8]:
# preview one of the tables
head(brunet)

,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,2,1,1,5,5,5,5,5,1,1,5,1,1,12
2,2,1,1,5,5,5,5,5,1,1,1,2,9,1
3,2,1,1,5,5,5,5,8,1,1,1,1,1,1
4,2,1,1,5,5,5,5,5,1,1,1,2,9,1
5,2,1,1,5,5,5,5,5,1,1,1,2,9,1
6,2,1,1,5,5,5,5,5,1,1,1,2,9,1


In [9]:
# Each number in the table refers to the group that each sample has been assigned to.
# Edit each value in the tables by adding "Group_" to each of the numbers so that they are strings and can be counted, not summed, by crosstab
brunet[,1:14] <- paste("Group", brunet[,1:14], sep = "_")
lee[,1:14] <- paste("Group", lee[,1:14], sep = "_")
hc[,1:14] <- paste("Group", hc[,1:14], sep = "_")
diana[,1:14] <- paste("Group", diana[,1:14], sep = "_")
km[,1:14] <- paste("Group", km[,1:14], sep = "_")
pam[,1:14] <- paste("Group", pam[,1:14], sep = "_")
ap[,1:14] <- paste("Group", ap[,1:14], sep = "_")
sc[,1:14] <- paste("Group", sc[,1:14], sep = "_")
gmm[,1:14] <- paste("Group", gmm[,1:14], sep = "_")
block[,1:14] <- paste("Group", block[,1:14], sep = "_")
som[,1:14] <- paste("Group", som[,1:14], sep = "_")
cmeans[,1:14] <- paste("Group", cmeans[,1:14], sep = "_")
head(brunet)

,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_5,Group_1,Group_1,Group_12
2,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
3,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_8,Group_1,Group_1,Group_1,Group_1,Group_1,Group_1
4,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
5,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
6,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1


In [10]:
# turn each of these matrices into dataframes
brunet <- as.data.frame(brunet)
lee <- as.data.frame(lee)
hc <- as.data.frame(hc)
diana <- as.data.frame(diana)
km <- as.data.frame(km)
pam <- as.data.frame(pam)
ap <- as.data.frame(ap)
sc <- as.data.frame(sc)
gmm <- as.data.frame(gmm)
block <- as.data.frame(block)
som <- as.data.frame(som)
cmeans <- as.data.frame(cmeans)

head(brunet)

,2,3,4,5,6,7,8,9,10,11,12,13,14,15
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_5,Group_1,Group_1,Group_12
2,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
3,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_8,Group_1,Group_1,Group_1,Group_1,Group_1,Group_1
4,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
5,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1
6,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1


In [11]:
# then add the is_known & vis columns back in to each df
brunet$is_known <- train$is_known
brunet$vis <- train$Vis

lee$is_known <- train$is_known
lee$vis <- train$Vis

hc$is_known <- train$is_known
hc$vis <- train$Vis

diana$is_known <- train$is_known
diana$vis <- train$Vis

km$is_known <- train$is_known
km$vis <- train$Vis

pam$is_known <- train$is_known
pam$vis <- train$Vis

ap$is_known <- train$is_known
ap$vis <- train$Vis

sc$is_known <- train$is_known
sc$vis <- train$Vis

gmm$is_known <- train$is_known
gmm$vis <- train$Vis

block$is_known <- train$is_known
block$vis <- train$Vis

som$is_known <- train$is_known
som$vis <- train$Vis

cmeans$is_known <- train$is_known
cmeans$vis <- train$Vis

head(brunet)

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,is_known,vis
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_5,Group_1,Group_1,Group_12,Guess,Gravel Cherts
2,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1,Guess,Gravel Cherts
3,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_8,Group_1,Group_1,Group_1,Group_1,Group_1,Group_1,Guess,Gravel Cherts
4,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1,Guess,Gravel Cherts
5,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1,Guess,Gravel Cherts
6,Group_2,Group_1,Group_1,Group_5,Group_5,Group_5,Group_5,Group_5,Group_1,Group_1,Group_1,Group_2,Group_9,Group_1,Guess,Gravel Cherts


In [12]:
# choose the best performing option by crosstab
# This one isn't good because most samples are in group 2
brunet_2_raw <- brunet[brunet$is_known == "Known",c(1,16)]
brunet_2_raw <- ftable(brunet_2_raw[])
brunet_2_raw <- as.data.frame(brunet_2_raw)
brunet_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_2,Alibates,22
Group_1,Edwards Plateau Chert,13
Group_2,Edwards Plateau Chert,59
Group_1,El Sauz Chert,4
Group_2,El Sauz Chert,271
Group_1,Knife River Flint,5
Group_2,Knife River Flint,7
Group_1,Oman Chert,1


In [13]:
# This one isn't good because it splits up ESC almost equally and puts the other groups with it
brunet_3_raw <- brunet[brunet$is_known == "Known",c(2,16)]
brunet_3_raw <- ftable(brunet_3_raw[])
brunet_3_raw <- as.data.frame(brunet_3_raw)
brunet_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,22
Group_2,Alibates,0
Group_3,Alibates,1
Group_1,Edwards Plateau Chert,53
Group_2,Edwards Plateau Chert,6
Group_3,Edwards Plateau Chert,13
Group_1,El Sauz Chert,147
Group_2,El Sauz Chert,121
Group_3,El Sauz Chert,7


In [14]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_4_raw <- brunet[brunet$is_known == "Known",c(3,16)]
brunet_4_raw <- ftable(brunet_4_raw[])
brunet_4_raw <- as.data.frame(brunet_4_raw)
brunet_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,21
Group_2,Alibates,0
Group_3,Alibates,1
Group_4,Alibates,1
Group_1,Edwards Plateau Chert,51
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,11
Group_4,Edwards Plateau Chert,10
Group_1,El Sauz Chert,144


In [15]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_5_raw <- brunet[brunet$is_known == "Known",c(4,16)]
brunet_5_raw <- ftable(brunet_5_raw[])
brunet_5_raw <- as.data.frame(brunet_5_raw)
brunet_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,0
Group_5,Alibates,21
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,12
Group_4,Edwards Plateau Chert,8


In [16]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_6_raw <- brunet[brunet$is_known == "Known",c(5,16)]
brunet_6_raw <- ftable(brunet_6_raw[])
brunet_6_raw <- as.data.frame(brunet_6_raw)
brunet_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,20
Group_6,Alibates,1
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,13


In [17]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_7_raw <- brunet[brunet$is_known == "Known",c(6,16)]
brunet_7_raw <- ftable(brunet_7_raw[])
brunet_7_raw <- as.data.frame(brunet_7_raw)
brunet_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,1
Group_4,Alibates,1
Group_5,Alibates,20
Group_7,Alibates,1
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,13


In [18]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_8_raw <- brunet[brunet$is_known == "Known",c(7,16)]
brunet_8_raw <- ftable(brunet_8_raw[])
brunet_8_raw <- as.data.frame(brunet_8_raw)
brunet_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,1
Group_3,Alibates,1
Group_4,Alibates,0
Group_5,Alibates,20
Group_6,Alibates,1
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,0


In [19]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_9_raw <- brunet[brunet$is_known == "Known",c(8,16)]
brunet_9_raw <- ftable(brunet_9_raw[])
brunet_9_raw <- as.data.frame(brunet_9_raw)
brunet_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,19
Group_6,Alibates,3
Group_7,Alibates,0
Group_8,Alibates,1
Group_1,Edwards Plateau Chert,0


In [20]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_10_raw <- brunet[brunet$is_known == "Known",c(9,16)]
brunet_10_raw <- ftable(brunet_10_raw[])
brunet_10_raw <- as.data.frame(brunet_10_raw)
brunet_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,17
Group_10,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,1
Group_4,Alibates,0
Group_5,Alibates,2
Group_6,Alibates,2
Group_7,Alibates,0
Group_9,Alibates,1


In [21]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_11_raw <- brunet[brunet$is_known == "Known",c(10,16)]
brunet_11_raw <- ftable(brunet_11_raw[])
brunet_11_raw <- as.data.frame(brunet_11_raw)
brunet_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,16
Group_10,Alibates,0
Group_11,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,5
Group_6,Alibates,1
Group_7,Alibates,0


In [22]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_12_raw <- brunet[brunet$is_known == "Known",c(11,16)]
brunet_12_raw <- ftable(brunet_12_raw[])
brunet_12_raw <- as.data.frame(brunet_12_raw)
brunet_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,12
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,9
Group_6,Alibates,0
Group_7,Alibates,1


In [23]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_13_raw <- brunet[brunet$is_known == "Known",c(12,16)]
brunet_13_raw <- ftable(brunet_13_raw[])
brunet_13_raw <- as.data.frame(brunet_13_raw)
brunet_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,7
Group_10,Alibates,1
Group_11,Alibates,0
Group_12,Alibates,0
Group_13,Alibates,1
Group_2,Alibates,10
Group_4,Alibates,0
Group_5,Alibates,2
Group_6,Alibates,2


In [24]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_14_raw <- brunet[brunet$is_known == "Known",c(13,16)]
brunet_14_raw <- ftable(brunet_14_raw[])
brunet_14_raw <- as.data.frame(brunet_14_raw)
brunet_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,13
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_13,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,4
Group_5,Alibates,0
Group_6,Alibates,0


In [25]:
# this one isn't good because it splits up ESC and puts the other groups with it
brunet_15_raw <- brunet[brunet$is_known == "Known",c(14,16)]
brunet_15_raw <- ftable(brunet_15_raw[])
brunet_15_raw <- as.data.frame(brunet_15_raw)
brunet_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,11
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,4
Group_13,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,1
Group_4,Alibates,6
Group_5,Alibates,0


In [26]:
# choose the best performing option by crosstab
# this one isn't good because it doesn't separate ESC from the rest
lee_2_raw <- lee[lee$is_known == "Known",c(1,16)]
lee_2_raw <- ftable(lee_2_raw[])
lee_2_raw <- as.data.frame(lee_2_raw)
lee_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_1,Edwards Plateau Chert,8
Group_2,Edwards Plateau Chert,64
Group_1,El Sauz Chert,3
Group_2,El Sauz Chert,272
Group_1,Knife River Flint,3
Group_2,Knife River Flint,9
Group_1,Oman Chert,1


In [27]:
# this one isn't good because it doesn't separate ESC from the rest
lee_3_raw <- lee[lee$is_known == "Known",c(2,16)]
lee_3_raw <- ftable(lee_3_raw[])
lee_3_raw <- as.data.frame(lee_3_raw)
lee_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_3,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,64
Group_3,Edwards Plateau Chert,8
Group_1,El Sauz Chert,13
Group_2,El Sauz Chert,259
Group_3,El Sauz Chert,3


In [28]:
# this one isn't good because it doesn't separate ESC from the rest
lee_4_raw <- lee[lee$is_known == "Known",c(3,16)]
lee_4_raw <- ftable(lee_4_raw[])
lee_4_raw <- as.data.frame(lee_4_raw)
lee_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_3,Alibates,0
Group_4,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,62
Group_3,Edwards Plateau Chert,2
Group_4,Edwards Plateau Chert,8
Group_1,El Sauz Chert,13


In [29]:
# this one isn't good because it doesn't separate ESC from the rest
lee_5_raw <- lee[lee$is_known == "Known",c(4,16)]
lee_5_raw <- ftable(lee_5_raw[])
lee_5_raw <- as.data.frame(lee_5_raw)
lee_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_1,Edwards Plateau Chert,7
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,2
Group_4,Edwards Plateau Chert,0


In [30]:
# this one isn't good because it doesn't separate ESC from the rest
lee_6_raw <- lee[lee$is_known == "Known",c(5,16)]
lee_6_raw <- ftable(lee_6_raw[])
lee_6_raw <- as.data.frame(lee_6_raw)
lee_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,2


In [31]:
# this one isn't good because it doesn't separate ESC from the rest
lee_7_raw <- lee[lee$is_known == "Known",c(6,16)]
lee_7_raw <- ftable(lee_7_raw[])
lee_7_raw <- as.data.frame(lee_7_raw)
lee_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_6,Alibates,0
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,2
Group_2,Edwards Plateau Chert,0


In [32]:
# this one isn't good because it doesn't separate ESC from the rest
lee_8_raw <- lee[lee$is_known == "Known",c(7,16)]
lee_8_raw <- ftable(lee_8_raw[])
lee_8_raw <- as.data.frame(lee_8_raw)
lee_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_5,Alibates,23
Group_7,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,0


In [33]:
# this one isn't good because it doesn't separate ESC from the rest
lee_9_raw <- lee[lee$is_known == "Known",c(8,16)]
lee_9_raw <- ftable(lee_9_raw[])
lee_9_raw <- as.data.frame(lee_9_raw)
lee_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_5,Alibates,23
Group_7,Alibates,0
Group_8,Alibates,0
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,1
Group_2,Edwards Plateau Chert,0


In [34]:
# this one isn't good because it doesn't separate ESC from the rest
lee_10_raw <- lee[lee$is_known == "Known",c(9,16)]
lee_10_raw <- ftable(lee_10_raw[])
lee_10_raw <- as.data.frame(lee_10_raw)
lee_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_2,Alibates,0
Group_5,Alibates,23
Group_6,Alibates,0
Group_7,Alibates,0
Group_9,Alibates,0
Group_10,Edwards Plateau Chert,2
Group_2,Edwards Plateau Chert,0
Group_5,Edwards Plateau Chert,66


In [35]:
# this one isn't good because it doesn't separate ESC from the rest
lee_11_raw <- lee[lee$is_known == "Known",c(10,16)]
lee_11_raw <- ftable(lee_11_raw[])
lee_11_raw <- as.data.frame(lee_11_raw)
lee_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,0
Group_2,Alibates,0
Group_5,Alibates,23
Group_6,Alibates,0
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_10,Edwards Plateau Chert,1


In [36]:
# this one isn't good because it doesn't separate ESC from the rest
lee_12_raw <- lee[lee$is_known == "Known",c(11,16)]
lee_12_raw <- ftable(lee_12_raw[])
lee_12_raw <- as.data.frame(lee_12_raw)
lee_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,1
Group_5,Alibates,22
Group_7,Alibates,0
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_10,Edwards Plateau Chert,0


In [37]:
# this one isn't good because it doesn't separate ESC from the rest
lee_13_raw <- lee[lee$is_known == "Known",c(12,16)]
lee_13_raw <- ftable(lee_13_raw[])
lee_13_raw <- as.data.frame(lee_13_raw)
lee_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_13,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_6,Alibates,0
Group_7,Alibates,0
Group_8,Alibates,0


In [38]:
# this one isn't good because it doesn't separate ESC from the rest
lee_14_raw <- lee[lee$is_known == "Known",c(13,16)]
lee_14_raw <- ftable(lee_14_raw[])
lee_14_raw <- as.data.frame(lee_14_raw)
lee_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_12,Alibates,0
Group_2,Alibates,0
Group_4,Alibates,23
Group_5,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,0
Group_9,Alibates,0
Group_12,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,0


In [39]:
# this one isn't good because it doesn't separate ESC from the rest
lee_15_raw <- lee[lee$is_known == "Known",c(14,16)]
lee_15_raw <- ftable(lee_15_raw[])
lee_15_raw <- as.data.frame(lee_15_raw)
lee_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_12,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,0
Group_4,Alibates,23
Group_6,Alibates,0
Group_9,Alibates,0
Group_10,Edwards Plateau Chert,0
Group_12,Edwards Plateau Chert,3


In [40]:
# choose the best performing option by crosstab
# there is only one group in this option
hc_2_raw <- hc[hc$is_known == "Known",c(1,16)]
hc_2_raw <- ftable(hc_2_raw[])
hc_2_raw <- as.data.frame(hc_2_raw)
hc_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [41]:
# there is only one group in this option
hc_3_raw <- hc[hc$is_known == "Known",c(2,16)]
hc_3_raw <- ftable(hc_3_raw[])
hc_3_raw <- as.data.frame(hc_3_raw)
hc_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [42]:
# there is only one group in this option
hc_4_raw <- hc[hc$is_known == "Known",c(3,16)]
hc_4_raw <- ftable(hc_4_raw[])
hc_4_raw <- as.data.frame(hc_4_raw)
hc_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [43]:
# there is only one group in this option
hc_5_raw <- hc[hc$is_known == "Known",c(4,16)]
hc_5_raw <- ftable(hc_5_raw[])
hc_5_raw <- as.data.frame(hc_5_raw)
hc_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [44]:
# there is only one group in this option
hc_6_raw <- hc[hc$is_known == "Known",c(5,16)]
hc_6_raw <- ftable(hc_6_raw[])
hc_6_raw <- as.data.frame(hc_6_raw)
hc_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [45]:
# this one isn't good because it doesn't separate ESC from the rest
hc_7_raw <- hc[hc$is_known == "Known",c(6,16)]
hc_7_raw <- ftable(hc_7_raw[])
hc_7_raw <- as.data.frame(hc_7_raw)
hc_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,2
Group_1,El Sauz Chert,275
Group_2,El Sauz Chert,0
Group_1,Knife River Flint,11
Group_2,Knife River Flint,1
Group_1,Oman Chert,21


In [46]:
# this one isn't good because it doesn't separate ESC from the rest
hc_8_raw <- hc[hc$is_known == "Known",c(7,16)]
hc_8_raw <- ftable(hc_8_raw[])
hc_8_raw <- as.data.frame(hc_8_raw)
hc_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,2
Group_8,Edwards Plateau Chert,0
Group_1,El Sauz Chert,275
Group_2,El Sauz Chert,0
Group_8,El Sauz Chert,0


In [47]:
# this one isn't good because it doesn't separate ESC from the rest
hc_9_raw <- hc[hc$is_known == "Known",c(8,16)]
hc_9_raw <- ftable(hc_9_raw[])
hc_9_raw <- as.data.frame(hc_9_raw)
hc_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_3,Alibates,0
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1
Group_9,Edwards Plateau Chert,0
Group_1,El Sauz Chert,275


In [48]:
# this one isn't good because it doesn't separate ESC from the rest
hc_10_raw <- hc[hc$is_known == "Known",c(9,16)]
hc_10_raw <- ftable(hc_10_raw[])
hc_10_raw <- as.data.frame(hc_10_raw)
hc_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_3,Alibates,0
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1
Group_9,Edwards Plateau Chert,0
Group_1,El Sauz Chert,275


In [49]:
# this one isn't good because it doesn't separate ESC from the rest
hc_11_raw <- hc[hc$is_known == "Known",c(10,16)]
hc_11_raw <- ftable(hc_11_raw[])
hc_11_raw <- as.data.frame(hc_11_raw)
hc_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_3,Alibates,0
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1
Group_9,Edwards Plateau Chert,0
Group_1,El Sauz Chert,275


In [50]:
# this one isn't good because it doesn't separate ESC from the rest
hc_12_raw <- hc[hc$is_known == "Known",c(11,16)]
hc_12_raw <- ftable(hc_12_raw[])
hc_12_raw <- as.data.frame(hc_12_raw)
hc_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_10,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_10,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1
Group_1,El Sauz Chert,275


In [51]:
# this one isn't good because it doesn't separate ESC from the rest
hc_13_raw <- hc[hc$is_known == "Known",c(12,16)]
hc_13_raw <- ftable(hc_13_raw[])
hc_13_raw <- as.data.frame(hc_13_raw)
hc_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_11,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_11,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1
Group_1,El Sauz Chert,275


In [52]:
# this one isn't good because it doesn't separate ESC from the rest
hc_14_raw <- hc[hc$is_known == "Known",c(13,16)]
hc_14_raw <- ftable(hc_14_raw[])
hc_14_raw <- as.data.frame(hc_14_raw)
hc_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_12,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,64
Group_12,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1


In [53]:
# this one isn't good because it doesn't separate ESC from the rest
hc_15_raw <- hc[hc$is_known == "Known",c(14,16)]
hc_15_raw <- ftable(hc_15_raw[])
hc_15_raw <- as.data.frame(hc_15_raw)
hc_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_13,Alibates,0
Group_2,Alibates,0
Group_4,Alibates,0
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,64
Group_13,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,1


In [54]:
# choose the best performing option by crosstab
# there is only one group in this option
diana_2_raw <- diana[diana$is_known == "Known",c(1,16)]
diana_2_raw <- ftable(diana_2_raw[])
diana_2_raw <- as.data.frame(diana_2_raw)
diana_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [55]:
# there is only one group in this option
diana_3_raw <- diana[diana$is_known == "Known",c(2,16)]
diana_3_raw <- ftable(diana_3_raw[])
diana_3_raw <- as.data.frame(diana_3_raw)
diana_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [56]:
# there is only one group in this option
diana_4_raw <- diana[diana$is_known == "Known",c(3,16)]
diana_4_raw <- ftable(diana_4_raw[])
diana_4_raw <- as.data.frame(diana_4_raw)
diana_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [57]:
# this one isn't good because it doesn't separate ESC from the rest
diana_5_raw <- diana[diana$is_known == "Known",c(4,16)]
diana_5_raw <- ftable(diana_5_raw[])
diana_5_raw <- as.data.frame(diana_5_raw)
diana_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,2
Group_1,El Sauz Chert,275
Group_2,El Sauz Chert,0
Group_1,Knife River Flint,11
Group_2,Knife River Flint,1
Group_1,Oman Chert,21


In [58]:
# this one isn't good because it doesn't separate ESC from the rest
diana_6_raw <- diana[diana$is_known == "Known",c(5,16)]
diana_6_raw <- ftable(diana_6_raw[])
diana_6_raw <- as.data.frame(diana_6_raw)
diana_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,2
Group_1,El Sauz Chert,275
Group_2,El Sauz Chert,0
Group_1,Knife River Flint,11
Group_2,Knife River Flint,1
Group_1,Oman Chert,21


In [59]:
# this one isn't good because it doesn't separate ESC from the rest
diana_7_raw <- diana[diana$is_known == "Known",c(6,16)]
diana_7_raw <- ftable(diana_7_raw[])
diana_7_raw <- as.data.frame(diana_7_raw)
diana_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,2
Group_1,El Sauz Chert,275
Group_2,El Sauz Chert,0
Group_1,Knife River Flint,11
Group_2,Knife River Flint,1
Group_1,Oman Chert,21


In [60]:
# this one isn't good because it doesn't separate ESC from the rest
diana_8_raw <- diana[diana$is_known == "Known",c(7,16)]
diana_8_raw <- ftable(diana_8_raw[])
diana_8_raw <- as.data.frame(diana_8_raw)
diana_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_3,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1
Group_1,El Sauz Chert,275
Group_2,El Sauz Chert,0
Group_3,El Sauz Chert,0


In [61]:
# this one isn't good because it doesn't separate ESC from the rest
diana_9_raw <- diana[diana$is_known == "Known",c(8,16)]
diana_9_raw <- ftable(diana_9_raw[])
diana_9_raw <- as.data.frame(diana_9_raw)
diana_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_1,Edwards Plateau Chert,64
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,6
Group_1,El Sauz Chert,272


In [62]:
# this one isn't good because it doesn't separate ESC from the rest
diana_10_raw <- diana[diana$is_known == "Known",c(9,16)]
diana_10_raw <- ftable(diana_10_raw[])
diana_10_raw <- as.data.frame(diana_10_raw)
diana_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_10,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_1,Edwards Plateau Chert,64
Group_10,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1


In [64]:
# this one isn't good because it doesn't separate ESC from the rest
diana_11_raw <- diana[diana$is_known == "Known",c(10,16)]
diana_11_raw <- ftable(diana_11_raw[])
diana_11_raw <- as.data.frame(diana_11_raw)
diana_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_10,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_1,Edwards Plateau Chert,64
Group_10,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1


In [65]:
# this one isn't good because it doesn't separate ESC from the rest
diana_12_raw <- diana[diana$is_known == "Known",c(11,16)]
diana_12_raw <- ftable(diana_12_raw[])
diana_12_raw <- as.data.frame(diana_12_raw)
diana_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_10,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_1,Edwards Plateau Chert,64
Group_10,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,1
Group_3,Edwards Plateau Chert,1


In [66]:
# this one isn't good because it doesn't separate ESC from the rest
diana_13_raw <- diana[diana$is_known == "Known",c(12,16)]
diana_13_raw <- ftable(diana_13_raw[])
diana_13_raw <- as.data.frame(diana_13_raw)
diana_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_11,Alibates,0
Group_2,Alibates,23
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_11,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,64


In [67]:
# this one isn't good because it doesn't separate ESC from the rest
diana_14_raw <- diana[diana$is_known == "Known",c(13,16)]
diana_14_raw <- ftable(diana_14_raw[])
diana_14_raw <- as.data.frame(diana_14_raw)
diana_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,23
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_12,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,64


In [68]:
# this one isn't good because it doesn't separate ESC from the rest
diana_15_raw <- diana[diana$is_known == "Known",c(14,16)]
diana_15_raw <- ftable(diana_15_raw[])
diana_15_raw <- as.data.frame(diana_15_raw)
diana_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_13,Alibates,0
Group_2,Alibates,23
Group_3,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_13,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,64


In [69]:
# choose the best performing option by crosstab
# there is only one group in this option
km_2_raw <- km[km$is_known == "Known",c(1,16)]
km_2_raw <- ftable(km_2_raw[])
km_2_raw <- as.data.frame(km_2_raw)
km_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_2,Alibates,23
Group_2,Edwards Plateau Chert,72
Group_2,El Sauz Chert,275
Group_2,Knife River Flint,12
Group_2,Oman Chert,21


In [70]:
# this one isn't good because it doesn't separate ESC from the rest
km_3_raw <- km[km$is_known == "Known",c(2,16)]
km_3_raw <- ftable(km_3_raw[])
km_3_raw <- as.data.frame(km_3_raw)
km_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_1,Edwards Plateau Chert,2
Group_2,Edwards Plateau Chert,70
Group_1,El Sauz Chert,0
Group_2,El Sauz Chert,275
Group_1,Knife River Flint,1
Group_2,Knife River Flint,11
Group_1,Oman Chert,0


In [71]:
# this one isn't good because it doesn't separate ESC from the rest
km_4_raw <- km[km$is_known == "Known",c(3,16)]
km_4_raw <- ftable(km_4_raw[])
km_4_raw <- as.data.frame(km_4_raw)
km_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_1,Edwards Plateau Chert,2
Group_2,Edwards Plateau Chert,70
Group_1,El Sauz Chert,0
Group_2,El Sauz Chert,275
Group_1,Knife River Flint,1
Group_2,Knife River Flint,11
Group_1,Oman Chert,0


In [72]:
# this one isn't good because it doesn't separate ESC from the rest
km_5_raw <- km[km$is_known == "Known",c(4,16)]
km_5_raw <- ftable(km_5_raw[])
km_5_raw <- as.data.frame(km_5_raw)
km_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,6
Group_2,Edwards Plateau Chert,64
Group_5,Edwards Plateau Chert,2
Group_1,El Sauz Chert,3
Group_2,El Sauz Chert,272
Group_5,El Sauz Chert,0


In [73]:
# this one isn't good because it doesn't separate ESC from the rest
km_6_raw <- km[km$is_known == "Known",c(5,16)]
km_6_raw <- ftable(km_6_raw[])
km_6_raw <- as.data.frame(km_6_raw)
km_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,23
Group_1,Edwards Plateau Chert,6
Group_2,Edwards Plateau Chert,0
Group_5,Edwards Plateau Chert,2
Group_6,Edwards Plateau Chert,64
Group_1,El Sauz Chert,3


In [74]:
# this one isn't good because it doesn't separate ESC from the rest
km_7_raw <- km[km$is_known == "Known",c(6,16)]
km_7_raw <- ftable(km_7_raw[])
km_7_raw <- as.data.frame(km_7_raw)
km_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,2
Group_2,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,21
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,10
Group_2,Edwards Plateau Chert,0
Group_5,Edwards Plateau Chert,4
Group_6,Edwards Plateau Chert,57


In [75]:
# this one isn't good because it doesn't separate ESC from the rest
km_8_raw <- km[km$is_known == "Known",c(7,16)]
km_8_raw <- ftable(km_8_raw[])
km_8_raw <- as.data.frame(km_8_raw)
km_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,2
Group_2,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,21
Group_7,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,9
Group_2,Edwards Plateau Chert,0
Group_5,Edwards Plateau Chert,5


In [76]:
# this one is better, but 63 ESC are still with the rest
# ESC = 194, Oman = 14
km_9_raw <- km[km$is_known == "Known",c(8,16)]
km_9_raw <- ftable(km_9_raw[])
km_9_raw <- as.data.frame(km_9_raw)
km_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,0
Group_5,Alibates,2
Group_6,Alibates,21
Group_7,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,5
Group_2,Edwards Plateau Chert,0


In [77]:
# this one is better, but 61 ESC are still with the rest
# ESC = 192, Oman = 14
km_10_raw <- km[km$is_known == "Known",c(9,16)]
km_10_raw <- ftable(km_10_raw[])
km_10_raw <- as.data.frame(km_10_raw)
km_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_2,Alibates,20
Group_3,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,3
Group_7,Alibates,0
Group_8,Alibates,0
Group_10,Edwards Plateau Chert,1
Group_2,Edwards Plateau Chert,56


In [78]:
# this one is better, but 46 ESC are still with the rest
# ESC = 191, Oman = 17
km_11_raw <- km[km$is_known == "Known",c(10,16)]
km_11_raw <- ftable(km_11_raw[])
km_11_raw <- as.data.frame(km_11_raw)
km_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_2,Alibates,18
Group_3,Alibates,0
Group_5,Alibates,1
Group_6,Alibates,4
Group_7,Alibates,0
Group_8,Alibates,0
Group_10,Edwards Plateau Chert,1


In [79]:
# this one is better, but 26 ESC are still with the rest
# ESC = 199, Oman = 17
km_12_raw <- km[km$is_known == "Known",c(11,16)]
km_12_raw <- ftable(km_12_raw[])
km_12_raw <- as.data.frame(km_12_raw)
km_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,17
Group_4,Alibates,0
Group_6,Alibates,5
Group_7,Alibates,1
Group_8,Alibates,0
Group_10,Edwards Plateau Chert,1


In [80]:
# this one is better, but 29 ESC are still with the rest
# ESC = 196, Oman = 11
km_13_raw <- km[km$is_known == "Known",c(12,16)]
km_13_raw <- ftable(km_13_raw[])
km_13_raw <- as.data.frame(km_13_raw)
km_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,15
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,7
Group_7,Alibates,1
Group_8,Alibates,0


In [81]:
# this one is better, but 34 ESC are still with the rest
# ESC = 187, Oman = 11
km_14_raw <- km[km$is_known == "Known",c(13,16)]
km_14_raw <- ftable(km_14_raw[])
km_14_raw <- as.data.frame(km_14_raw)
km_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,14
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,7
Group_7,Alibates,2


In [82]:
# this one is better, but 36 ESC are still with the rest
# ESC = 185, Oman = 11
km_15_raw <- km[km$is_known == "Known",c(14,16)]
km_15_raw <- ftable(km_15_raw[])
km_15_raw <- as.data.frame(km_15_raw)
km_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_14,Alibates,0
Group_15,Alibates,4
Group_2,Alibates,14
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,4


In [83]:
# choose the best performing option by crosstab
# this one isn't good because it doesn't separate ESC from the rest
pam_2_raw <- pam[pam$is_known == "Known",c(1,16)]
pam_2_raw <- ftable(pam_2_raw[])
pam_2_raw <- as.data.frame(pam_2_raw)
pam_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,2
Group_1,El Sauz Chert,275
Group_2,El Sauz Chert,0
Group_1,Knife River Flint,11
Group_2,Knife River Flint,1
Group_1,Oman Chert,21


In [84]:
# this one isn't good because it doesn't separate ESC from the rest
pam_3_raw <- pam[pam$is_known == "Known",c(2,16)]
pam_3_raw <- ftable(pam_3_raw[])
pam_3_raw <- as.data.frame(pam_3_raw)
pam_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,2
Group_1,El Sauz Chert,275
Group_2,El Sauz Chert,0
Group_1,Knife River Flint,11
Group_2,Knife River Flint,1
Group_1,Oman Chert,21


In [85]:
# this one isn't good because it doesn't separate ESC from the rest
pam_4_raw <- pam[pam$is_known == "Known",c(3,16)]
pam_4_raw <- ftable(pam_4_raw[])
pam_4_raw <- as.data.frame(pam_4_raw)
pam_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,22
Group_2,Alibates,0
Group_3,Alibates,1
Group_1,Edwards Plateau Chert,60
Group_2,Edwards Plateau Chert,2
Group_3,Edwards Plateau Chert,10
Group_1,El Sauz Chert,267
Group_2,El Sauz Chert,0
Group_3,El Sauz Chert,8


In [86]:
# only 8 ESC are with the rest, but it doesn't sort out oman
pam_5_raw <- pam[pam$is_known == "Known",c(4,16)]
pam_5_raw <- ftable(pam_5_raw[])
pam_5_raw <- as.data.frame(pam_5_raw)
pam_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,22
Group_3,Alibates,0
Group_4,Alibates,1
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,60
Group_3,Edwards Plateau Chert,2
Group_4,Edwards Plateau Chert,10
Group_1,El Sauz Chert,259


In [87]:
# only 8 ESC are with the rest, but it doesn't sort out oman
pam_6_raw <- pam[pam$is_known == "Known",c(5,16)]
pam_6_raw <- ftable(pam_6_raw[])
pam_6_raw <- as.data.frame(pam_6_raw)
pam_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,22
Group_3,Alibates,0
Group_4,Alibates,1
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,60
Group_3,Edwards Plateau Chert,2
Group_4,Edwards Plateau Chert,10
Group_1,El Sauz Chert,259


In [88]:
# this one puts 16 ESC with the rest, but it sorts out oman better than the previous option
# ESC = 235, Oman = 17
pam_7_raw <- pam[pam$is_known == "Known",c(6,16)]
pam_7_raw <- ftable(pam_7_raw[])
pam_7_raw <- as.data.frame(pam_7_raw)
pam_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,8
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,2
Group_4,Edwards Plateau Chert,19


In [89]:
# this one puts 16 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 235, Oman = 19
pam_8_raw <- pam[pam$is_known == "Known",c(7,16)]
pam_8_raw <- ftable(pam_8_raw[])
pam_8_raw <- as.data.frame(pam_8_raw)
pam_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,7
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,1


In [90]:
# this one puts 16 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 235, Oman = 19
pam_9_raw <- pam[pam$is_known == "Known",c(8,16)]
pam_9_raw <- ftable(pam_9_raw[])
pam_9_raw <- as.data.frame(pam_9_raw)
pam_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,7
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,1


In [91]:
# this one puts 16 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 235, Oman = 19
pam_10_raw <- pam[pam$is_known == "Known",c(9,16)]
pam_10_raw <- ftable(pam_10_raw[])
pam_10_raw <- as.data.frame(pam_10_raw)
pam_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,7
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,1


In [92]:
# this one puts 16 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 211, Oman = 19
pam_11_raw <- pam[pam$is_known == "Known",c(10,16)]
pam_11_raw <- ftable(pam_11_raw[])
pam_11_raw <- as.data.frame(pam_11_raw)
pam_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,1
Group_6,Alibates,7
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,45


In [93]:
# this one puts 14 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 213, Oman = 17
pam_12_raw <- pam[pam$is_known == "Known",c(11,16)]
pam_12_raw <- ftable(pam_12_raw[])
pam_12_raw <- as.data.frame(pam_12_raw)
pam_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_2,Alibates,14
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,6
Group_6,Alibates,2
Group_8,Alibates,1
Group_1,Edwards Plateau Chert,0


In [94]:
# this one puts 17 ESC with the rest, and it sorts out oman better than the previous few options
# ESC = 205, Oman = 13
pam_13_raw <- pam[pam$is_known == "Known",c(12,16)]
pam_13_raw <- ftable(pam_13_raw[])
pam_13_raw <- as.data.frame(pam_13_raw)
pam_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_11,Alibates,0
Group_2,Alibates,12
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,6
Group_6,Alibates,4
Group_7,Alibates,0
Group_9,Alibates,1


In [95]:
# this one puts 36 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 205, Oman = 13
pam_14_raw <- pam[pam$is_known == "Known",c(13,16)]
pam_14_raw <- ftable(pam_14_raw[])
pam_14_raw <- as.data.frame(pam_14_raw)
pam_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,12
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,3
Group_6,Alibates,1
Group_7,Alibates,0


In [96]:
# this one puts 36 ESC with the rest, and it sorts out oman better than pam_6
# ESC = 205, Oman = 13
pam_15_raw <- pam[pam$is_known == "Known",c(14,16)]
pam_15_raw <- ftable(pam_15_raw[])
pam_15_raw <- as.data.frame(pam_15_raw)
pam_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,12
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,3
Group_6,Alibates,1
Group_7,Alibates,0


In [97]:
# choose the best performing option by crosstab
# this one isn't good because it doesn't separate ESC from the rest
ap_2_raw <- ap[ap$is_known == "Known",c(1,16)]
ap_2_raw <- ftable(ap_2_raw[])
ap_2_raw <- as.data.frame(ap_2_raw)
ap_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_2,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_2,Edwards Plateau Chert,2
Group_1,El Sauz Chert,275
Group_2,El Sauz Chert,0
Group_1,Knife River Flint,11
Group_2,Knife River Flint,1
Group_1,Oman Chert,21


In [98]:
# this one isn't good because it doesn't separate ESC from the rest
ap_3_raw <- ap[ap$is_known == "Known",c(2,16)]
ap_3_raw <- ftable(ap_3_raw[])
ap_3_raw <- as.data.frame(ap_3_raw)
ap_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_3,Alibates,0
Group_1,Edwards Plateau Chert,70
Group_3,Edwards Plateau Chert,2
Group_1,El Sauz Chert,275
Group_3,El Sauz Chert,0
Group_1,Knife River Flint,11
Group_3,Knife River Flint,1
Group_1,Oman Chert,21


In [99]:
# this one isn't good because it doesn't separate ESC from the rest
ap_4_raw <- ap[ap$is_known == "Known",c(3,16)]
ap_4_raw <- ftable(ap_4_raw[])
ap_4_raw <- as.data.frame(ap_4_raw)
ap_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,22
Group_3,Alibates,0
Group_4,Alibates,1
Group_1,Edwards Plateau Chert,61
Group_3,Edwards Plateau Chert,2
Group_4,Edwards Plateau Chert,9
Group_1,El Sauz Chert,269
Group_3,El Sauz Chert,0
Group_4,El Sauz Chert,6


In [100]:
# this one is much better, it only puts 8 ESC with the rest, but it doesn't sort out Oman
ap_5_raw <- ap[ap$is_known == "Known",c(4,16)]
ap_5_raw <- ftable(ap_5_raw[])
ap_5_raw <- as.data.frame(ap_5_raw)
ap_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,22
Group_2,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,1
Group_1,Edwards Plateau Chert,61
Group_2,Edwards Plateau Chert,0
Group_4,Edwards Plateau Chert,2
Group_5,Edwards Plateau Chert,9
Group_1,El Sauz Chert,8


In [101]:
# this one is good, it separates both ESC and most of Oman
# ESC = 247, Oman = 15
ap_6_raw <- ap[ap$is_known == "Known",c(5,16)]
ap_6_raw <- ftable(ap_6_raw[])
ap_6_raw <- as.data.frame(ap_6_raw)
ap_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,5
Group_3,Alibates,18
Group_4,Alibates,0
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,14
Group_3,Edwards Plateau Chert,50
Group_4,Edwards Plateau Chert,6


In [102]:
# this one is good, it separates both ESC and most of Oman
# ESC = 248, Oman = 14
ap_7_raw <- ap[ap$is_known == "Known",c(6,16)]
ap_7_raw <- ftable(ap_7_raw[])
ap_7_raw <- as.data.frame(ap_7_raw)
ap_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,20
Group_6,Alibates,3
Group_1,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,5
Group_4,Edwards Plateau Chert,1
Group_5,Edwards Plateau Chert,53


In [103]:
# this one is good, it separates both ESC and most of Oman
# ESC = 240, Oman = 14
ap_8_raw <- ap[ap$is_known == "Known",c(7,16)]
ap_8_raw <- ftable(ap_8_raw[])
ap_8_raw <- as.data.frame(ap_8_raw)
ap_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,0
Group_6,Alibates,7
Group_7,Alibates,0
Group_8,Alibates,14
Group_1,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,9
Group_4,Edwards Plateau Chert,5


In [104]:
# this one is good, it separates both ESC and most of Oman
# ESC = 235, Oman = 18
ap_9_raw <- ap[ap$is_known == "Known",c(8,16)]
ap_9_raw <- ftable(ap_9_raw[])
ap_9_raw <- as.data.frame(ap_9_raw)
ap_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,7
Group_2,Alibates,0
Group_4,Alibates,0
Group_6,Alibates,15
Group_7,Alibates,0
Group_9,Alibates,1
Group_1,Edwards Plateau Chert,17
Group_2,Edwards Plateau Chert,0
Group_4,Edwards Plateau Chert,2


In [105]:
# this one is good, it separates both ESC and most of Oman
# ESC = 235, Oman = 17
ap_10_raw <- ap[ap$is_known == "Known",c(9,16)]
ap_10_raw <- ftable(ap_10_raw[])
ap_10_raw <- as.data.frame(ap_10_raw)
ap_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,7
Group_2,Alibates,0
Group_3,Alibates,1
Group_5,Alibates,0
Group_7,Alibates,0
Group_8,Alibates,15
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,17
Group_2,Edwards Plateau Chert,0


In [106]:
# this one is good, it separates both ESC and most of Oman
# ESC = 235, Oman = 17
ap_11_raw <- ap[ap$is_known == "Known",c(10,16)]
ap_11_raw <- ftable(ap_11_raw[])
ap_11_raw <- as.data.frame(ap_11_raw)
ap_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,7
Group_2,Alibates,0
Group_3,Alibates,1
Group_5,Alibates,0
Group_7,Alibates,0
Group_8,Alibates,15
Group_9,Alibates,0
Group_1,Edwards Plateau Chert,17
Group_2,Edwards Plateau Chert,0


In [107]:
# this one is good, it separates both ESC and most of Oman
# ESC = 194 & 50, Oman = 16
ap_12_raw <- ap[ap$is_known == "Known",c(11,16)]
ap_12_raw <- ftable(ap_12_raw[])
ap_12_raw <- as.data.frame(ap_12_raw)
ap_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_10,Alibates,0
Group_11,Alibates,14
Group_3,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,0
Group_9,Alibates,8
Group_1,Edwards Plateau Chert,5


In [108]:
# this one is good, it separates both ESC and most of Oman
# ESC = 205, Oman = 13
ap_13_raw <- ap[ap$is_known == "Known",c(12,16)]
ap_13_raw <- ftable(ap_13_raw[])
ap_13_raw <- as.data.frame(ap_13_raw)
ap_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,3
Group_12,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,12
Group_4,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,0


In [109]:
# this one is good, it separates both ESC and most of Oman
# ESC = 205, Oman = 13
ap_14_raw <- ap[ap$is_known == "Known",c(13,16)]
ap_14_raw <- ftable(ap_14_raw[])
ap_14_raw <- as.data.frame(ap_14_raw)
ap_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,3
Group_12,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,12
Group_4,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,0


In [110]:
# this one is good, it separates both ESC and most of Oman
# ESC = 205, Oman = 13
ap_15_raw <- ap[ap$is_known == "Known",c(14,16)]
ap_15_raw <- ftable(ap_15_raw[])
ap_15_raw <- as.data.frame(ap_15_raw)
ap_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,3
Group_12,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,12
Group_4,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,0


In [111]:
# choose the best performing option by crosstab
# there is only one group in this option
sc_2_raw <- sc[sc$is_known == "Known",c(1,16)]
sc_2_raw <- ftable(sc_2_raw[])
sc_2_raw <- as.data.frame(sc_2_raw)
sc_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [112]:
# there is only one group in this option
sc_3_raw <- sc[sc$is_known == "Known",c(2,16)]
sc_3_raw <- ftable(sc_3_raw[])
sc_3_raw <- as.data.frame(sc_3_raw)
sc_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,23
Group_1,Edwards Plateau Chert,72
Group_1,El Sauz Chert,275
Group_1,Knife River Flint,12
Group_1,Oman Chert,21


In [113]:
# this one isn't good because it doesn't separate ESC from the rest
sc_4_raw <- sc[sc$is_known == "Known",c(3,16)]
sc_4_raw <- ftable(sc_4_raw[])
sc_4_raw <- as.data.frame(sc_4_raw)
sc_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,7
Group_2,Alibates,0
Group_3,Alibates,1
Group_4,Alibates,15
Group_1,Edwards Plateau Chert,15
Group_2,Edwards Plateau Chert,5
Group_3,Edwards Plateau Chert,6
Group_4,Edwards Plateau Chert,46
Group_1,El Sauz Chert,30


In [114]:
# this one isn't good because it doesn't separate ESC from the rest
sc_5_raw <- sc[sc$is_known == "Known",c(4,16)]
sc_5_raw <- ftable(sc_5_raw[])
sc_5_raw <- as.data.frame(sc_5_raw)
sc_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,3
Group_3,Alibates,14
Group_4,Alibates,6
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,1
Group_2,Edwards Plateau Chert,12
Group_3,Edwards Plateau Chert,36
Group_4,Edwards Plateau Chert,16


In [115]:
# this one is good, it separates both ESC and most of Oman
# ESC = 159, Oman = 15
sc_6_raw <- sc[sc$is_known == "Known",c(5,16)]
sc_6_raw <- ftable(sc_6_raw[])
sc_6_raw <- as.data.frame(sc_6_raw)
sc_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,5
Group_2,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,16
Group_1,Edwards Plateau Chert,18
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,11


In [116]:
# this one is good, it separates both ESC and most of Oman
# ESC = 158, Oman = 21
sc_7_raw <- sc[sc$is_known == "Known",c(6,16)]
sc_7_raw <- ftable(sc_7_raw[])
sc_7_raw <- as.data.frame(sc_7_raw)
sc_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,4
Group_3,Alibates,17
Group_4,Alibates,1
Group_5,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,1
Group_1,Edwards Plateau Chert,0
Group_2,Edwards Plateau Chert,18


In [117]:
# this one is good, it separates both ESC and most of Oman
# ESC = 118 & 90, Oman = 18
sc_8_raw <- sc[sc$is_known == "Known",c(7,16)]
sc_8_raw <- ftable(sc_8_raw[])
sc_8_raw <- as.data.frame(sc_8_raw)
sc_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,6
Group_2,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,13
Group_5,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,1
Group_8,Alibates,1
Group_1,Edwards Plateau Chert,15


In [118]:
# this one is good, it separates both ESC and most of Oman
# ESC = 119 & 83, Oman = 17
sc_9_raw <- sc[sc$is_known == "Known",c(8,16)]
sc_9_raw <- ftable(sc_9_raw[])
sc_9_raw <- as.data.frame(sc_9_raw)
sc_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,1
Group_4,Alibates,1
Group_5,Alibates,2
Group_6,Alibates,0
Group_7,Alibates,0
Group_8,Alibates,13
Group_9,Alibates,6


In [119]:
# this one is good, it separates both ESC and most of Oman
# ESC = 110 & 81, Oman = 21
sc_10_raw <- sc[sc$is_known == "Known",c(9,16)]
sc_10_raw <- ftable(sc_10_raw[])
sc_10_raw <- as.data.frame(sc_10_raw)
sc_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,12
Group_2,Alibates,0
Group_3,Alibates,3
Group_4,Alibates,1
Group_5,Alibates,0
Group_6,Alibates,0
Group_7,Alibates,1
Group_8,Alibates,0


In [120]:
# this one is good, it separates both ESC and most of Oman
# ESC = 99 & 62 & 66, Oman = 21
sc_11_raw <- sc[sc$is_known == "Known",c(10,16)]
sc_11_raw <- ftable(sc_11_raw[])
sc_11_raw <- as.data.frame(sc_11_raw)
sc_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,1
Group_11,Alibates,1
Group_2,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,8
Group_5,Alibates,0
Group_6,Alibates,4
Group_7,Alibates,1


In [121]:
# this one is good, it separates both ESC and most of Oman
# ESC = 105, 63, 66, Oman = 21
sc_12_raw <- sc[sc$is_known == "Known",c(11,16)]
sc_12_raw <- ftable(sc_12_raw[])
sc_12_raw <- as.data.frame(sc_12_raw)
sc_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,5
Group_11,Alibates,0
Group_12,Alibates,8
Group_2,Alibates,1
Group_3,Alibates,0
Group_4,Alibates,6
Group_5,Alibates,0
Group_6,Alibates,0


In [122]:
# this one separates into too many groups
sc_13_raw <- sc[sc$is_known == "Known",c(12,16)]
sc_13_raw <- ftable(sc_13_raw[])
sc_13_raw <- as.data.frame(sc_13_raw)
sc_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,3
Group_12,Alibates,0
Group_13,Alibates,4
Group_2,Alibates,0
Group_3,Alibates,1
Group_4,Alibates,6
Group_5,Alibates,2


In [123]:
# this one separates into too many groups
sc_14_raw <- sc[sc$is_known == "Known",c(13,16)]
sc_14_raw <- ftable(sc_14_raw[])
sc_14_raw <- as.data.frame(sc_14_raw)
sc_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,7
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,3
Group_13,Alibates,0
Group_14,Alibates,0
Group_2,Alibates,5
Group_3,Alibates,2
Group_4,Alibates,0


In [124]:
# this one separates into too many groups
sc_15_raw <- sc[sc$is_known == "Known",c(14,16)]
sc_15_raw <- ftable(sc_15_raw[])
sc_15_raw <- as.data.frame(sc_15_raw)
sc_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,4
Group_11,Alibates,0
Group_12,Alibates,3
Group_13,Alibates,5
Group_14,Alibates,0
Group_15,Alibates,3
Group_2,Alibates,0
Group_3,Alibates,0


In [125]:
# choose the best performing option by crosstab
# Oman isn't separated out of the rest, but ESC is good
gmm_2_raw <- gmm[gmm$is_known == "Known",c(1,16)]
gmm_2_raw <- ftable(gmm_2_raw[])
gmm_2_raw <- as.data.frame(gmm_2_raw)
gmm_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,3
Group_2,Alibates,20
Group_1,Edwards Plateau Chert,20
Group_2,Edwards Plateau Chert,52
Group_1,El Sauz Chert,38
Group_2,El Sauz Chert,237
Group_1,Knife River Flint,5
Group_2,Knife River Flint,7
Group_1,Oman Chert,20


In [126]:
# there is alot of overlap in the groups
gmm_3_raw <- gmm[gmm$is_known == "Known",c(2,16)]
gmm_3_raw <- ftable(gmm_3_raw[])
gmm_3_raw <- as.data.frame(gmm_3_raw)
gmm_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,5
Group_2,Alibates,18
Group_3,Alibates,0
Group_1,Edwards Plateau Chert,22
Group_2,Edwards Plateau Chert,49
Group_3,Edwards Plateau Chert,1
Group_1,El Sauz Chert,90
Group_2,El Sauz Chert,181
Group_3,El Sauz Chert,4


In [127]:
# this one is good, it separates both ESC and most of Oman
# ESC = 226, Oman = 18
gmm_4_raw <- gmm[gmm$is_known == "Known",c(3,16)]
gmm_4_raw <- ftable(gmm_4_raw[])
gmm_4_raw <- as.data.frame(gmm_4_raw)
gmm_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,3
Group_2,Alibates,16
Group_3,Alibates,0
Group_4,Alibates,4
Group_1,Edwards Plateau Chert,1
Group_2,Edwards Plateau Chert,50
Group_3,Edwards Plateau Chert,0
Group_4,Edwards Plateau Chert,21
Group_1,El Sauz Chert,226


In [128]:
# this one is good, it separates both ESC and most of Oman
# ESC = 236, Oman = 13
gmm_5_raw <- gmm[gmm$is_known == "Known",c(4,16)]
gmm_5_raw <- ftable(gmm_5_raw[])
gmm_5_raw <- as.data.frame(gmm_5_raw)
gmm_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,5
Group_2,Alibates,16
Group_3,Alibates,0
Group_4,Alibates,2
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,21
Group_2,Edwards Plateau Chert,46
Group_3,Edwards Plateau Chert,0
Group_4,Edwards Plateau Chert,0


In [129]:
gmm_6_raw <- gmm[gmm$is_known == "Known",c(5,16)]
gmm_6_raw <- ftable(gmm_6_raw[])
gmm_6_raw <- as.data.frame(gmm_6_raw)
gmm_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,5
Group_2,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,2
Group_5,Alibates,16
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,22
Group_2,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,0


In [130]:
gmm_7_raw <- gmm[gmm$is_known == "Known",c(6,16)]
gmm_7_raw <- ftable(gmm_7_raw[])
gmm_7_raw <- as.data.frame(gmm_7_raw)
gmm_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,4
Group_2,Alibates,9
Group_3,Alibates,2
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,8
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,12
Group_2,Edwards Plateau Chert,26


In [131]:
gmm_8_raw <- gmm[gmm$is_known == "Known",c(7,16)]
gmm_8_raw <- ftable(gmm_8_raw[])
gmm_8_raw <- as.data.frame(gmm_8_raw)
gmm_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_2,Alibates,8
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,4
Group_6,Alibates,3
Group_7,Alibates,7
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,8


In [132]:
gmm_9_raw <- gmm[gmm$is_known == "Known",c(8,16)]
gmm_9_raw <- ftable(gmm_9_raw[])
gmm_9_raw <- as.data.frame(gmm_9_raw)
gmm_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_2,Alibates,3
Group_3,Alibates,3
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,11
Group_7,Alibates,1
Group_8,Alibates,4
Group_9,Alibates,0


In [133]:
gmm_10_raw <- gmm[gmm$is_known == "Known",c(9,16)]
gmm_10_raw <- ftable(gmm_10_raw[])
gmm_10_raw <- as.data.frame(gmm_10_raw)
gmm_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,4
Group_10,Alibates,0
Group_2,Alibates,8
Group_3,Alibates,2
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,3
Group_7,Alibates,3
Group_8,Alibates,0


In [134]:
gmm_11_raw <- gmm[gmm$is_known == "Known",c(10,16)]
gmm_11_raw <- ftable(gmm_11_raw[])
gmm_11_raw <- as.data.frame(gmm_11_raw)
gmm_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,4
Group_11,Alibates,0
Group_2,Alibates,5
Group_3,Alibates,0
Group_4,Alibates,1
Group_5,Alibates,0
Group_6,Alibates,1
Group_7,Alibates,7


In [135]:
gmm_12_raw <- gmm[gmm$is_known == "Known",c(11,16)]
gmm_12_raw <- ftable(gmm_12_raw[])
gmm_12_raw <- as.data.frame(gmm_12_raw)
gmm_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,4
Group_11,Alibates,0
Group_2,Alibates,3
Group_3,Alibates,2
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,1
Group_7,Alibates,9


In [136]:
gmm_13_raw <- gmm[gmm$is_known == "Known",c(12,16)]
gmm_13_raw <- ftable(gmm_13_raw[])
gmm_13_raw <- as.data.frame(gmm_13_raw)
gmm_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,4
Group_11,Alibates,3
Group_12,Alibates,0
Group_2,Alibates,0
Group_3,Alibates,2
Group_4,Alibates,5
Group_5,Alibates,0
Group_6,Alibates,1


In [137]:
# this option isn't good, it splits up ESC too much
gmm_14_raw <- gmm[gmm$is_known == "Known",c(13,16)]
gmm_14_raw <- ftable(gmm_14_raw[])
gmm_14_raw <- as.data.frame(gmm_14_raw)
gmm_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,5
Group_12,Alibates,3
Group_13,Alibates,0
Group_2,Alibates,8
Group_3,Alibates,3
Group_4,Alibates,0
Group_5,Alibates,0


In [138]:
# this option isn't good, it splits up ESC too much
gmm_15_raw <- gmm[gmm$is_known == "Known",c(14,16)]
gmm_15_raw <- ftable(gmm_15_raw[])
gmm_15_raw <- as.data.frame(gmm_15_raw)
gmm_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,2
Group_13,Alibates,1
Group_14,Alibates,0
Group_2,Alibates,3
Group_3,Alibates,3
Group_4,Alibates,2


In [139]:
# choose the best performing option by crosstab
# this one isn't good because it doesn't separate ESC from the rest
block_2_raw <- block[block$is_known == "Known",c(1,16)]
block_2_raw <- ftable(block_2_raw[])
block_2_raw <- as.data.frame(block_2_raw)
block_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,20
Group_2,Alibates,3
Group_1,Edwards Plateau Chert,55
Group_2,Edwards Plateau Chert,17
Group_1,El Sauz Chert,270
Group_2,El Sauz Chert,5
Group_1,Knife River Flint,10
Group_2,Knife River Flint,2
Group_1,Oman Chert,17


In [140]:
block_3_raw <- block[block$is_known == "Known",c(2,16)]
block_3_raw <- ftable(block_3_raw[])
block_3_raw <- as.data.frame(block_3_raw)
block_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,5
Group_2,Alibates,16
Group_3,Alibates,2
Group_1,Edwards Plateau Chert,11
Group_2,Edwards Plateau Chert,47
Group_3,Edwards Plateau Chert,14
Group_1,El Sauz Chert,245
Group_2,El Sauz Chert,26
Group_3,El Sauz Chert,4


In [141]:
# this one isn't good because it doesn't separate ESC from the rest
block_4_raw <- block[block$is_known == "Known",c(3,16)]
block_4_raw <- ftable(block_4_raw[])
block_4_raw <- as.data.frame(block_4_raw)
block_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,9
Group_2,Alibates,3
Group_3,Alibates,0
Group_4,Alibates,11
Group_1,Edwards Plateau Chert,12
Group_2,Edwards Plateau Chert,13
Group_3,Edwards Plateau Chert,3
Group_4,Edwards Plateau Chert,44
Group_1,El Sauz Chert,112


In [142]:
block_5_raw <- block[block$is_known == "Known",c(4,16)]
block_5_raw <- ftable(block_5_raw[])
block_5_raw <- as.data.frame(block_5_raw)
block_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,14
Group_3,Alibates,0
Group_4,Alibates,6
Group_5,Alibates,3
Group_1,Edwards Plateau Chert,1
Group_2,Edwards Plateau Chert,45
Group_3,Edwards Plateau Chert,3
Group_4,Edwards Plateau Chert,9


In [143]:
# there is only one group in this option
block_6_raw <- block[block$is_known == "Known",c(5,16)]
block_6_raw <- ftable(block_6_raw[])
block_6_raw <- as.data.frame(block_6_raw)
block_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_NA,Alibates,23
Group_NA,Edwards Plateau Chert,72
Group_NA,El Sauz Chert,275
Group_NA,Knife River Flint,12
Group_NA,Oman Chert,21


In [144]:
block_7_raw <- block[block$is_known == "Known",c(6,16)]
block_7_raw <- ftable(block_7_raw[])
block_7_raw <- as.data.frame(block_7_raw)
block_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,2
Group_2,Alibates,0
Group_3,Alibates,14
Group_4,Alibates,2
Group_5,Alibates,3
Group_6,Alibates,2
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,2
Group_2,Edwards Plateau Chert,0


In [145]:
block_8_raw <- block[block$is_known == "Known",c(7,16)]
block_8_raw <- ftable(block_8_raw[])
block_8_raw <- as.data.frame(block_8_raw)
block_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,3
Group_2,Alibates,2
Group_3,Alibates,1
Group_4,Alibates,14
Group_5,Alibates,1
Group_6,Alibates,2
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,9
Group_2,Edwards Plateau Chert,14


In [146]:
# there is only one group in this option
block_9_raw <- block[block$is_known == "Known",c(8,16)]
block_9_raw <- ftable(block_9_raw[])
block_9_raw <- as.data.frame(block_9_raw)
block_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_NA,Alibates,23
Group_NA,Edwards Plateau Chert,72
Group_NA,El Sauz Chert,275
Group_NA,Knife River Flint,12
Group_NA,Oman Chert,21


In [147]:
block_10_raw <- block[block$is_known == "Known",c(9,16)]
block_10_raw <- ftable(block_10_raw[])
block_10_raw <- as.data.frame(block_10_raw)
block_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,1
Group_2,Alibates,0
Group_3,Alibates,14
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,2
Group_7,Alibates,1
Group_8,Alibates,3


In [148]:
# there is only one group in this option
block_11_raw <- block[block$is_known == "Known",c(10,16)]
block_11_raw <- ftable(block_11_raw[])
block_11_raw <- as.data.frame(block_11_raw)
block_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_NA,Alibates,23
Group_NA,Edwards Plateau Chert,72
Group_NA,El Sauz Chert,275
Group_NA,Knife River Flint,12
Group_NA,Oman Chert,21


In [149]:
# there is only one group in this option
block_12_raw <- block[block$is_known == "Known",c(11,16)]
block_12_raw <- ftable(block_12_raw[])
block_12_raw <- as.data.frame(block_12_raw)
block_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_NA,Alibates,23
Group_NA,Edwards Plateau Chert,72
Group_NA,El Sauz Chert,275
Group_NA,Knife River Flint,12
Group_NA,Oman Chert,21


In [150]:
# there is only one group in this option
block_13_raw <- block[block$is_known == "Known",c(12,16)]
block_13_raw <- ftable(block_13_raw[])
block_13_raw <- as.data.frame(block_13_raw)
block_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_NA,Alibates,23
Group_NA,Edwards Plateau Chert,72
Group_NA,El Sauz Chert,275
Group_NA,Knife River Flint,12
Group_NA,Oman Chert,21


In [151]:
# there is only one group in this option
block_14_raw <- block[block$is_known == "Known",c(13,16)]
block_14_raw <- ftable(block_14_raw[])
block_14_raw <- as.data.frame(block_14_raw)
block_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_NA,Alibates,23
Group_NA,Edwards Plateau Chert,72
Group_NA,El Sauz Chert,275
Group_NA,Knife River Flint,12
Group_NA,Oman Chert,21


In [152]:
# there is only one group in this option
block_15_raw <- block[block$is_known == "Known",c(14,16)]
block_15_raw <- ftable(block_15_raw[])
block_15_raw <- as.data.frame(block_15_raw)
block_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_NA,Alibates,23
Group_NA,Edwards Plateau Chert,72
Group_NA,El Sauz Chert,275
Group_NA,Knife River Flint,12
Group_NA,Oman Chert,21


In [153]:
# choose the best performing option by crosstab
# there is only one group in this option
som_2_raw <- som[som$is_known == "Known",c(1,16)]
som_2_raw <- ftable(som_2_raw[])
som_2_raw <- as.data.frame(som_2_raw)
som_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_2,Alibates,23
Group_2,Edwards Plateau Chert,72
Group_2,El Sauz Chert,275
Group_2,Knife River Flint,12
Group_2,Oman Chert,21


In [154]:
# there is only one group in this option
som_3_raw <- som[som$is_known == "Known",c(2,16)]
som_3_raw <- ftable(som_3_raw[])
som_3_raw <- as.data.frame(som_3_raw)
som_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_3,Alibates,23
Group_3,Edwards Plateau Chert,72
Group_3,El Sauz Chert,275
Group_3,Knife River Flint,12
Group_3,Oman Chert,21


In [155]:
# this one isn't good because it doesn't separate ESC from the rest
som_4_raw <- som[som$is_known == "Known",c(3,16)]
som_4_raw <- ftable(som_4_raw[])
som_4_raw <- as.data.frame(som_4_raw)
som_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_3,Alibates,0
Group_4,Alibates,23
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,71
Group_3,El Sauz Chert,0
Group_4,El Sauz Chert,275
Group_3,Knife River Flint,0
Group_4,Knife River Flint,12
Group_3,Oman Chert,0


In [156]:
# this one isn't good because it doesn't separate ESC from the rest
som_5_raw <- som[som$is_known == "Known",c(4,16)]
som_5_raw <- ftable(som_5_raw[])
som_5_raw <- as.data.frame(som_5_raw)
som_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_3,Alibates,0
Group_4,Alibates,23
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,71
Group_3,El Sauz Chert,0
Group_4,El Sauz Chert,275
Group_3,Knife River Flint,0
Group_4,Knife River Flint,12
Group_3,Oman Chert,0


In [157]:
# this one isn't good because it doesn't separate ESC from the rest
som_6_raw <- som[som$is_known == "Known",c(5,16)]
som_6_raw <- ftable(som_6_raw[])
som_6_raw <- as.data.frame(som_6_raw)
som_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_3,Alibates,0
Group_4,Alibates,23
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,71
Group_3,El Sauz Chert,0
Group_4,El Sauz Chert,275
Group_3,Knife River Flint,0
Group_4,Knife River Flint,12
Group_3,Oman Chert,0


In [158]:
# this one isn't good because it doesn't separate ESC from the rest
som_7_raw <- som[som$is_known == "Known",c(6,16)]
som_7_raw <- ftable(som_7_raw[])
som_7_raw <- as.data.frame(som_7_raw)
som_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_3,Alibates,0
Group_4,Alibates,23
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,71
Group_3,El Sauz Chert,0
Group_4,El Sauz Chert,275
Group_3,Knife River Flint,0
Group_4,Knife River Flint,12
Group_3,Oman Chert,0


In [159]:
# this one isn't good because it doesn't separate ESC from the rest
som_8_raw <- som[som$is_known == "Known",c(7,16)]
som_8_raw <- ftable(som_8_raw[])
som_8_raw <- as.data.frame(som_8_raw)
som_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,3
Group_5,Edwards Plateau Chert,68
Group_3,El Sauz Chert,0
Group_4,El Sauz Chert,0
Group_5,El Sauz Chert,275


In [160]:
# this one isn't good because it doesn't separate ESC from the rest
som_9_raw <- som[som$is_known == "Known",c(8,16)]
som_9_raw <- ftable(som_9_raw[])
som_9_raw <- as.data.frame(som_9_raw)
som_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_8,Alibates,0
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,1
Group_5,Edwards Plateau Chert,68
Group_8,Edwards Plateau Chert,2
Group_3,El Sauz Chert,0


In [161]:
# this one isn't good because it doesn't separate ESC from the rest
som_10_raw <- som[som$is_known == "Known",c(9,16)]
som_10_raw <- ftable(som_10_raw[])
som_10_raw <- as.data.frame(som_10_raw)
som_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_8,Alibates,0
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,1
Group_5,Edwards Plateau Chert,68
Group_8,Edwards Plateau Chert,2
Group_3,El Sauz Chert,0


In [162]:
# this one isn't good because it doesn't separate ESC from the rest
som_11_raw <- som[som$is_known == "Known",c(10,16)]
som_11_raw <- ftable(som_11_raw[])
som_11_raw <- as.data.frame(som_11_raw)
som_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_8,Alibates,0
Group_10,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,1
Group_5,Edwards Plateau Chert,68


In [163]:
# this one isn't good because it doesn't separate ESC from the rest
som_12_raw <- som[som$is_known == "Known",c(11,16)]
som_12_raw <- ftable(som_12_raw[])
som_12_raw <- as.data.frame(som_12_raw)
som_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_11,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_9,Alibates,0
Group_11,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,1
Group_5,Edwards Plateau Chert,68


In [164]:
# this one isn't good because it doesn't separate ESC from the rest
som_13_raw <- som[som$is_known == "Known",c(12,16)]
som_13_raw <- ftable(som_13_raw[])
som_13_raw <- as.data.frame(som_13_raw)
som_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_11,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,23
Group_9,Alibates,0
Group_11,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,1
Group_4,Edwards Plateau Chert,1
Group_5,Edwards Plateau Chert,68


In [165]:
# this one isn't good because it doesn't separate ESC from the rest
som_14_raw <- som[som$is_known == "Known",c(13,16)]
som_14_raw <- ftable(som_14_raw[])
som_14_raw <- as.data.frame(som_14_raw)
som_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,23
Group_12,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,0
Group_9,Alibates,0
Group_10,Edwards Plateau Chert,65
Group_12,Edwards Plateau Chert,0
Group_3,Edwards Plateau Chert,1


In [166]:
# this one isn't good because it doesn't separate ESC from the rest
som_15_raw <- som[som$is_known == "Known",c(14,16)]
som_15_raw <- ftable(som_15_raw[])
som_15_raw <- as.data.frame(som_15_raw)
som_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_10,Alibates,0
Group_11,Alibates,23
Group_13,Alibates,0
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,0
Group_10,Edwards Plateau Chert,2
Group_11,Edwards Plateau Chert,65
Group_13,Edwards Plateau Chert,0


In [167]:
# choose the best performing option by crosstab
# there is only one group in this option
cmeans_2_raw <- cmeans[cmeans$is_known == "Known",c(1,16)]
cmeans_2_raw <- ftable(cmeans_2_raw[])
cmeans_2_raw <- as.data.frame(cmeans_2_raw)
cmeans_2_raw

X2,vis,Freq
<fct>,<fct>,<int>
Group_2,Alibates,23
Group_2,Edwards Plateau Chert,72
Group_2,El Sauz Chert,275
Group_2,Knife River Flint,12
Group_2,Oman Chert,21


In [168]:
# this one isn't good because it doesn't separate ESC from the rest
cmeans_3_raw <- cmeans[cmeans$is_known == "Known",c(2,16)]
cmeans_3_raw <- ftable(cmeans_3_raw[])
cmeans_3_raw <- as.data.frame(cmeans_3_raw)
cmeans_3_raw

X3,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_1,Edwards Plateau Chert,2
Group_2,Edwards Plateau Chert,70
Group_1,El Sauz Chert,0
Group_2,El Sauz Chert,275
Group_1,Knife River Flint,1
Group_2,Knife River Flint,11
Group_1,Oman Chert,0


In [169]:
# this one isn't good because it doesn't separate ESC from the rest
cmeans_4_raw <- cmeans[cmeans$is_known == "Known",c(3,16)]
cmeans_4_raw <- ftable(cmeans_4_raw[])
cmeans_4_raw <- as.data.frame(cmeans_4_raw)
cmeans_4_raw

X4,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_1,Edwards Plateau Chert,2
Group_2,Edwards Plateau Chert,70
Group_1,El Sauz Chert,0
Group_2,El Sauz Chert,275
Group_1,Knife River Flint,1
Group_2,Knife River Flint,11
Group_1,Oman Chert,0


In [170]:
# this one isn't good because it doesn't separate ESC from the rest
cmeans_5_raw <- cmeans[cmeans$is_known == "Known",c(4,16)]
cmeans_5_raw <- ftable(cmeans_5_raw[])
cmeans_5_raw <- as.data.frame(cmeans_5_raw)
cmeans_5_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,7
Group_2,Edwards Plateau Chert,64
Group_5,Edwards Plateau Chert,1
Group_1,El Sauz Chert,3
Group_2,El Sauz Chert,272
Group_5,El Sauz Chert,0


In [171]:
# this one isn't good because it doesn't separate ESC from the rest
cmeans_6_raw <- cmeans[cmeans$is_known == "Known",c(5,16)]
cmeans_6_raw <- ftable(cmeans_6_raw[])
cmeans_6_raw <- as.data.frame(cmeans_6_raw)
cmeans_6_raw

X6,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,23
Group_5,Alibates,0
Group_6,Alibates,0
Group_1,Edwards Plateau Chert,7
Group_2,Edwards Plateau Chert,64
Group_5,Edwards Plateau Chert,1
Group_6,Edwards Plateau Chert,0
Group_1,El Sauz Chert,3


In [172]:
# this one isn't good because it doesn't separate ESC from the rest
cmeans_7_raw <- cmeans[cmeans$is_known == "Known",c(6,16)]
cmeans_7_raw <- ftable(cmeans_7_raw[])
cmeans_7_raw <- as.data.frame(cmeans_7_raw)
cmeans_7_raw

X7,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,21
Group_5,Alibates,2
Group_6,Alibates,0
Group_7,Alibates,0
Group_1,Edwards Plateau Chert,4
Group_2,Edwards Plateau Chert,57
Group_5,Edwards Plateau Chert,10
Group_6,Edwards Plateau Chert,0


In [173]:
# this one isn't good because it doesn't separate ESC from the rest
cmeans_8_raw <- cmeans[cmeans$is_known == "Known",c(7,16)]
cmeans_8_raw <- ftable(cmeans_8_raw[])
cmeans_8_raw <- as.data.frame(cmeans_8_raw)
cmeans_8_raw

X8,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,20
Group_5,Alibates,3
Group_6,Alibates,0
Group_7,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,1
Group_2,Edwards Plateau Chert,55
Group_5,Edwards Plateau Chert,10


In [174]:
# this one isn't good because it doesn't separate ESC from the rest
cmeans_9_raw <- cmeans[cmeans$is_known == "Known",c(8,16)]
cmeans_9_raw <- ftable(cmeans_9_raw[])
cmeans_9_raw <- as.data.frame(cmeans_9_raw)
cmeans_9_raw

X9,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_2,Alibates,21
Group_3,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,2
Group_7,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,4
Group_2,Edwards Plateau Chert,57


In [175]:
cmeans_10_raw <- cmeans[cmeans$is_known == "Known",c(9,16)]
cmeans_10_raw <- ftable(cmeans_10_raw[])
cmeans_10_raw <- as.data.frame(cmeans_10_raw)
cmeans_10_raw

X10,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_2,Alibates,21
Group_3,Alibates,0
Group_5,Alibates,1
Group_6,Alibates,1
Group_7,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,3


In [176]:
cmeans_11_raw <- cmeans[cmeans$is_known == "Known",c(10,16)]
cmeans_11_raw <- ftable(cmeans_11_raw[])
cmeans_11_raw <- as.data.frame(cmeans_11_raw)
cmeans_11_raw

X11,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,0
Group_2,Alibates,21
Group_5,Alibates,0
Group_6,Alibates,2
Group_7,Alibates,0
Group_8,Alibates,0
Group_1,Edwards Plateau Chert,4


In [177]:
cmeans_12_raw <- cmeans[cmeans$is_known == "Known",c(11,16)]
cmeans_12_raw <- ftable(cmeans_12_raw[])
cmeans_12_raw <- as.data.frame(cmeans_12_raw)
cmeans_12_raw

X12,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_2,Alibates,15
Group_5,Alibates,0
Group_6,Alibates,7
Group_7,Alibates,0
Group_8,Alibates,0


In [178]:
cmeans_13_raw <- cmeans[cmeans$is_known == "Known",c(12,16)]
cmeans_13_raw <- ftable(cmeans_13_raw[])
cmeans_13_raw <- as.data.frame(cmeans_13_raw)
cmeans_13_raw

X13,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_10,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,0
Group_13,Alibates,0
Group_2,Alibates,18
Group_5,Alibates,0
Group_6,Alibates,5
Group_7,Alibates,0


In [179]:
cmeans_14_raw <- cmeans[cmeans$is_known == "Known",c(13,16)]
cmeans_14_raw <- ftable(cmeans_14_raw[])
cmeans_14_raw <- as.data.frame(cmeans_14_raw)
cmeans_14_raw

X14,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,1
Group_11,Alibates,0
Group_12,Alibates,0
Group_13,Alibates,0
Group_2,Alibates,15
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,0
Group_6,Alibates,7


In [180]:
# this one splits ESC & Oman into too many groups
cmeans_15_raw <- cmeans[cmeans$is_known == "Known",c(14,16)]
cmeans_15_raw <- ftable(cmeans_15_raw[])
cmeans_15_raw <- as.data.frame(cmeans_15_raw)
cmeans_15_raw

X15,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,0
Group_11,Alibates,0
Group_12,Alibates,2
Group_13,Alibates,0
Group_15,Alibates,0
Group_2,Alibates,14
Group_3,Alibates,0
Group_4,Alibates,0
Group_5,Alibates,0
